# FastVLM Training with Qwen3-0.6B (EPOCH-based)

This notebook trains FastVLM using **EPOCHS** instead of steps.

- **LLM Backbone**: Qwen3-0.6B
- **Vision Encoder**: FastViTHD (MobileCLIP)
- **Dataset**: 5CD-AI/Viet-multimodal-open-r1-8k-verified
- **Output**: Push to HuggingFace: beyoru/Belle-VLM

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers>=4.51.0,<5.0.0
!pip install -q torch>=2.1.0 torchvision>=0.16.0
!pip install -q accelerate>=0.26.0 peft>=0.10.0
!pip install -q bitsandbytes>=0.43.0
!pip install -q datasets pillow einops timm>=0.9.0
!pip install -q deepspeed sentencepiece
!pip install -q huggingface_hub safetensors!pip install -q open_clip_torch
!pip install -q git+https://github.com/apple/ml-mobileclip.git


In [ ]:
# Clone the FastVLM repository
!git clone https://github.com/Hert4/ml-fastvlm-v2.git
%cd ml-fastvlm-v2

## 2. Configuration (EPOCH-based)

In [ ]:
import os
import json
import torch
from pathlib import Path

# ============================================
# CONFIGURATION - EPOCH BASED
# ============================================
CONFIG = {
    # Model
    "llm_model": "Qwen/Qwen3-0.6B",
    "vision_tower": "apple/MobileCLIP2-S4",
    
    # Dataset
    "dataset_name": "5CD-AI/Viet-multimodal-open-r1-8k-verified",
    "image_column": "image",
    "question_column": "vi_problem",
    "answer_column": "vi_solution",
    
    # Training - EPOCHS
    "output_dir": "./outputs/fastvlm-qwen3-0.6b-vietnamese",
    "num_train_epochs": 3,              # <-- Number of epochs
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "learning_rate": 2e-5,
    "warmup_ratio": 0.03,               # <-- 3% of total steps for warmup
    "lr_scheduler_type": "cosine",
    "bf16": True,
    "model_max_length": 2048,
    
    # LoRA
    "use_lora": True,
    "lora_r": 64,
    "lora_alpha": 128,
    "lora_dropout": 0.05,
    
    # Save checkpoints - EPOCH based
    "save_strategy": "epoch",           # <-- Save every epoch
    "save_total_limit": 2,
    
    # HuggingFace
    "hf_repo": "beyoru/Belle-VLM",
    "hf_token": os.environ.get("HF_TOKEN", ""),
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

print("Configuration (EPOCH-based):")
for k, v in CONFIG.items():
    if k != "hf_token":
        print(f"  {k}: {v}")

# Estimate training
effective_batch = CONFIG["per_device_train_batch_size"] * CONFIG["gradient_accumulation_steps"]
print(f"\n Training info:")
print(f"  Effective batch size: {effective_batch}")
print(f"  Epochs: {CONFIG['num_train_epochs']}")
print(f"  Warmup: {CONFIG['warmup_ratio']*100:.0f}% of total steps")
print(f"  Save: every epoch")

In [ ]:
# Login to HuggingFace
from huggingface_hub import login

if CONFIG["hf_token"]:
    login(token=CONFIG["hf_token"])
    print("Logged in to HuggingFace!")
else:
    print("HF_TOKEN not set. Set it in Kaggle Secrets.")

## 3. Load and Prepare Dataset

In [ ]:
from datasets import load_dataset
from PIL import Image

# Load dataset
print(f"Loading dataset: {CONFIG['dataset_name']}")
dataset = load_dataset(CONFIG["dataset_name"], split="train")

print(f"\nDataset info:")
print(f"  Total samples: {len(dataset)}")
print(f"  Columns: {dataset.column_names}")

# Estimate steps per epoch
effective_batch = CONFIG["per_device_train_batch_size"] * CONFIG["gradient_accumulation_steps"]
steps_per_epoch = len(dataset) // effective_batch
total_steps = steps_per_epoch * CONFIG["num_train_epochs"]
warmup_steps = int(total_steps * CONFIG["warmup_ratio"])

print(f"\n Training estimates:")
print(f"  Steps per epoch: ~{steps_per_epoch}")
print(f"  Total steps: ~{total_steps}")
print(f"  Warmup steps: ~{warmup_steps}")

In [ ]:
# Display sample image
from IPython.display import display

sample = dataset[0]
sample_image = sample['image']
if isinstance(sample_image, Image.Image):
    display(sample_image.resize((400, 400)))
print(f"\nQuestion: {sample['vi_problem'][:200]}...")
print(f"Answer: {sample['vi_solution'][:200]}...")

In [ ]:
import json
import os
from tqdm import tqdm

def convert_dataset_to_llava_format(dataset, output_dir, image_folder):
    """Convert dataset to LLaVA training format."""
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    llava_data = []
    
    for idx, sample in enumerate(tqdm(dataset, desc="Converting dataset")):
        # Save image
        image_filename = f"{idx:06d}.jpg"
        image_path = os.path.join(image_folder, image_filename)
        
        img = sample['image']
        if isinstance(img, Image.Image):
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(image_path, 'JPEG', quality=95)
        
        # Create conversation
        question = sample['vi_problem'].strip()
        answer = sample['vi_solution'].strip()
        
        # Truncate very long solutions
        if len(answer) > 4096:
            answer = answer[:4096] + "..."
        
        llava_sample = {
            "id": str(sample.get('id', idx)),
            "image": image_filename,
            "conversations": [
                {"from": "human", "value": f"<image>\n{question}"},
                {"from": "gpt", "value": answer}
            ]
        }
        llava_data.append(llava_sample)
    
    # Save JSON
    json_path = os.path.join(output_dir, "train_data.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(llava_data, f, ensure_ascii=False, indent=2)
    
    print(f"\nDataset converted: {len(llava_data)} samples")
    return json_path, image_folder

# Convert dataset
DATA_DIR = "./data"
IMAGE_FOLDER = os.path.join(DATA_DIR, "images")

json_path, image_folder = convert_dataset_to_llava_format(dataset, DATA_DIR, IMAGE_FOLDER)

## 4. Train the Model (EPOCH-based)

In [ ]:
# Check GPU
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Run training using EPOCHS
import subprocess
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check TF32 support
def supports_tf32():
    if torch.cuda.is_available():
        return torch.cuda.get_device_capability()[0] >= 8
    return False

use_tf32 = supports_tf32()

# Build command - EPOCH BASED
cmd = [
    "python", "-m", "llava.train.train_qwen",
    f"--model_name_or_path={CONFIG['llm_model']}",
    "--version=qwen_3",
    f"--data_path={json_path}",
    f"--image_folder={image_folder}",
    f"--vision_tower={CONFIG['vision_tower']}",
    "--mm_projector_type=mlp2x_gelu",
    "--mm_vision_select_layer=-2",
    "--mm_use_im_start_end=False",
    "--mm_use_im_patch_token=False",
    "--image_aspect_ratio=pad",
    f"--output_dir={CONFIG['output_dir']}",
    
    # EPOCH CONFIG
    f"--num_train_epochs={CONFIG['num_train_epochs']}",
    f"--warmup_ratio={CONFIG['warmup_ratio']}",
    f"--save_strategy={CONFIG['save_strategy']}",
    f"--save_total_limit={CONFIG['save_total_limit']}",
    
    f"--per_device_train_batch_size={CONFIG['per_device_train_batch_size']}",
    f"--gradient_accumulation_steps={CONFIG['gradient_accumulation_steps']}",
    f"--learning_rate={CONFIG['learning_rate']}",
    "--weight_decay=0.0",
    f"--lr_scheduler_type={CONFIG['lr_scheduler_type']}",
    "--logging_steps=10",
    f"--model_max_length={CONFIG['model_max_length']}",
    "--gradient_checkpointing=True",
    "--dataloader_num_workers=4",
    "--lazy_preprocess=True",
    "--bf16=True",
    "--report_to=none",
    
    # LoRA
    "--lora_enable=True",
    f"--lora_r={CONFIG['lora_r']}",
    f"--lora_alpha={CONFIG['lora_alpha']}",
    f"--lora_dropout={CONFIG['lora_dropout']}",
]

if use_tf32:
    cmd.append("--tf32=True")
else:
    cmd.append("--tf32=False")

print(f"Starting training for {CONFIG['num_train_epochs']} epochs...")
print(f"Checkpoints will be saved every epoch")

env = os.environ.copy()
env["CUDA_VISIBLE_DEVICES"] = "0"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1,
    env=env,
)

for line in process.stdout:
    print(line, end='')

process.wait()
print(f"\n Training completed with return code: {process.returncode}")

## 5. Check Training Output

In [ ]:
# Check training output
import os

OUTPUT_DIR = CONFIG["output_dir"]
print(f"Checking: {OUTPUT_DIR}\n")

# Check for checkpoint folders
checkpoints = []
if os.path.exists(OUTPUT_DIR):
    for item in os.listdir(OUTPUT_DIR):
        item_path = os.path.join(OUTPUT_DIR, item)
        if os.path.isdir(item_path) and "checkpoint" in item:
            checkpoints.append(item_path)

# List all files
all_files = []
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path) / 1024 / 1024
        all_files.append((path, size))
        print(f"  {path} ({size:.2f} MB)")

print(f"\nTotal files: {len(all_files)}")
print(f"Checkpoints found: {checkpoints}")

## 6. Merge LoRA + mm_projector

In [ ]:
# ============================================
# MERGE LORA + MM_PROJECTOR INTO ONE MODEL
# ============================================
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os

OUTPUT_DIR = CONFIG["output_dir"]
MERGED_DIR = os.path.join(OUTPUT_DIR, "merged")
os.makedirs(MERGED_DIR, exist_ok=True)

# Load base model
print("Loading base Qwen3 model...")
base_model = AutoModelForCausalLM.from_pretrained(
    CONFIG["llm_model"],
    torch_dtype=torch.float16,
    device_map="cpu",
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG["llm_model"], trust_remote_code=True)
tokenizer.save_pretrained(MERGED_DIR)

# Load and merge LoRA
print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

print("Merging LoRA weights...")
model = model.merge_and_unload()

# Load mm_projector from non_lora_trainables.bin
print("\nLoading mm_projector weights...")
non_lora_path = os.path.join(OUTPUT_DIR, "non_lora_trainables.bin")
non_lora_weights = torch.load(non_lora_path, map_location="cpu")

# Get model state dict
state_dict = model.state_dict()

# Add mm_projector weights to state dict
mm_count = 0
for k, v in non_lora_weights.items():
    if 'mm_projector' in k:
        # Clean key: remove prefixes
        clean_key = k
        for prefix in ['base_model.model.', 'model.']:
            if clean_key.startswith(prefix):
                clean_key = clean_key[len(prefix):]
        
        # Add "model." prefix for LlavaQwen3 structure
        if not clean_key.startswith('model.'):
            clean_key = 'model.' + clean_key
        
        state_dict[clean_key] = v.to(torch.float16)
        print(f"  Added: {clean_key} {v.shape}")
        mm_count += 1

print(f"\nAdded {mm_count} mm_projector tensors")

# Save merged model with mm_projector included
print("\nSaving complete model...")
model.save_pretrained(MERGED_DIR, state_dict=state_dict, safe_serialization=True)

print(f"\n Model saved to: {MERGED_DIR}")
print("   mm_projector is NOW INCLUDED in model.safetensors!")

In [ ]:
# ============================================
# VERIFY mm_projector IS IN model.safetensors
# ============================================
from safetensors import safe_open
import os

safetensor_path = os.path.join(MERGED_DIR, "model.safetensors")

if os.path.exists(safetensor_path):
    print(f" {safetensor_path}")
    print(f"   Size: {os.path.getsize(safetensor_path) / 1024 / 1024:.2f} MB\n")
    
    with safe_open(safetensor_path, framework="pt") as f:
        keys = f.keys()
        
        # Find mm_projector keys
        mm_keys = [k for k in keys if 'mm_projector' in k]
        
        if mm_keys:
            print(" mm_projector FOUND in model.safetensors:")
            for k in mm_keys:
                tensor = f.get_tensor(k)
                print(f"   {k}: {tensor.shape}")
        else:
            print(" mm_projector NOT FOUND!")
else:
    print(f" File not found: {safetensor_path}")

In [ ]:
# ============================================
# UPDATE CONFIG.JSON
# ============================================
import json

src_config = os.path.join(OUTPUT_DIR, "config.json")
dst_config = os.path.join(MERGED_DIR, "config.json")

if os.path.exists(src_config):
    with open(src_config, 'r') as f:
        config_data = json.load(f)
    
    # Add auto_map for HuggingFace
    config_data["auto_map"] = {
        "AutoConfig": "configuration_llava_qwen.LlavaQwen3Config",
        "AutoModelForCausalLM": "modeling_llava_qwen.LlavaQwen3ForCausalLM"
    }
    config_data["architectures"] = ["LlavaQwen3ForCausalLM"]
    
    with open(dst_config, 'w') as f:
        json.dump(config_data, f, indent=2)
    
    print(" Updated config.json with auto_map")

# Verify files
print(f"\n Files in {MERGED_DIR}:")
for f in sorted(os.listdir(MERGED_DIR)):
    size = os.path.getsize(os.path.join(MERGED_DIR, f)) / 1024 / 1024
    print(f"  {f} ({size:.2f} MB)")

## 7. Upload to HuggingFace

In [ ]:
# Create model card
model_card = f"""---
license: apache-2.0
language:
- vi
- en
tags:
- vision-language-model
- vlm
- qwen3
- fastvlm
- vietnamese
base_model: {CONFIG['llm_model']}
datasets:
- {CONFIG['dataset_name']}
---

# Belle-VLM: Vietnamese Vision Language Model

## Model Description

Belle-VLM is a Vision Language Model trained for Vietnamese multimodal reasoning tasks.

### Architecture
- **LLM Backbone**: Qwen3-0.6B
- **Vision Encoder**: FastViTHD (MobileCLIP)
- **Projector**: MLP 2-layer (3072 -> 1024)

### Training
- **Dataset**: {CONFIG['dataset_name']}
- **Method**: LoRA fine-tuning
- **Epochs**: {CONFIG['num_train_epochs']}
- **Learning Rate**: {CONFIG['learning_rate']}

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "{CONFIG['hf_repo']}",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("{CONFIG['hf_repo']}", trust_remote_code=True)
```

## Training Details

| Parameter | Value |
|-----------|-------|
| Base Model | {CONFIG['llm_model']} |
| Vision Tower | {CONFIG['vision_tower']} |
| LoRA Rank | {CONFIG['lora_r']} |
| LoRA Alpha | {CONFIG['lora_alpha']} |
| Batch Size | {CONFIG['per_device_train_batch_size']} x {CONFIG['gradient_accumulation_steps']} |
| Epochs | {CONFIG['num_train_epochs']} |

## License

Apache 2.0
"""

readme_path = os.path.join(MERGED_DIR, "README.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(model_card)

print(" Model card created!")

In [ ]:
# ============================================
# CREATE CUSTOM CODE FILES FOR HUGGINGFACE
# ============================================
# These files are REQUIRED for trust_remote_code=True

# 1. configuration_llava_qwen.py
config_code = '''# Configuration for LLaVA Qwen3 model
from transformers import Qwen2Config
try:
    from transformers import Qwen3Config
    QWEN3_AVAILABLE = True
except ImportError:
    QWEN3_AVAILABLE = False
    Qwen3Config = Qwen2Config

class LlavaQwen3Config(Qwen3Config if QWEN3_AVAILABLE else Qwen2Config):
    model_type = "llava_qwen3"
    def __init__(self, mm_vision_tower=None, mm_hidden_size=None, mm_projector_type="mlp2x_gelu",
                 mm_vision_select_layer=-2, mm_vision_select_feature="patch", mm_patch_merge_type="flat",
                 mm_use_im_start_end=False, mm_use_im_patch_token=False, image_aspect_ratio="pad", **kwargs):
        super().__init__(**kwargs)
        self.mm_vision_tower = mm_vision_tower
        self.mm_hidden_size = mm_hidden_size
        self.mm_projector_type = mm_projector_type
        self.mm_vision_select_layer = mm_vision_select_layer
        self.mm_vision_select_feature = mm_vision_select_feature
        self.mm_patch_merge_type = mm_patch_merge_type
        self.mm_use_im_start_end = mm_use_im_start_end
        self.mm_use_im_patch_token = mm_use_im_patch_token
        self.image_aspect_ratio = image_aspect_ratio
'''

with open("/tmp/configuration_llava_qwen.py", "w") as f:
    f.write(config_code)

print("Created configuration_llava_qwen.py")

In [ ]:
# 2. modeling_llava_qwen.py
modeling_code = '''# LLaVA Qwen3 Model
from typing import List, Optional, Tuple, Union
from abc import ABC, abstractmethod
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModelForCausalLM, Qwen2Config, Qwen2Model, Qwen2ForCausalLM
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers.generation.utils import GenerateOutput

try:
    from transformers import Qwen3Config, Qwen3Model, Qwen3ForCausalLM
    QWEN3_AVAILABLE = True
except ImportError:
    QWEN3_AVAILABLE = False
    Qwen3Config, Qwen3Model, Qwen3ForCausalLM = Qwen2Config, Qwen2Model, Qwen2ForCausalLM

from .configuration_llava_qwen import LlavaQwen3Config

IGNORE_INDEX = -100
IMAGE_TOKEN_INDEX = -200

def build_vision_projector(config):
    projector_type = getattr(config, "mm_projector_type", "mlp2x_gelu")
    mm_hidden_size = getattr(config, "mm_hidden_size", 3072)
    hidden_size = config.hidden_size
    if projector_type == "mlp2x_gelu":
        return nn.Sequential(nn.Linear(mm_hidden_size, hidden_size), nn.GELU(), nn.Linear(hidden_size, hidden_size))
    return nn.Linear(mm_hidden_size, hidden_size)

class MobileCLIPVisionTower(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.is_loaded = False
        self.image_processor = None
        self.vision_tower = None
        self.hidden_size = getattr(config, "mm_hidden_size", 3072)
        try:
            self.image_size = int(getattr(config, "mm_vision_tower", "mobileclip_l_384").split("_")[-1])
        except:
            self.image_size = 384

    def load_model(self, **kwargs):
        if self.is_loaded:
            return
        try:
            import timm
            from transformers import CLIPImageProcessor
            self.vision_tower = timm.create_model("fastvit_mci2.apple_mclip", pretrained=True, num_classes=0)
            self.vision_tower.eval()
            self.image_processor = CLIPImageProcessor(size={"shortest_edge": self.image_size},
                crop_size={"height": self.image_size, "width": self.image_size}, do_normalize=True,
                image_mean=[0.48145466, 0.4578275, 0.40821073], image_std=[0.26862954, 0.26130258, 0.27577711])
            self.is_loaded = True
            print("MobileCLIP loaded!")
        except Exception as e:
            print(f"Could not load MobileCLIP: {e}")
            self.is_loaded = True

    def forward(self, images):
        if not self.is_loaded:
            self.load_model()
        with torch.no_grad():
            features = self.vision_tower.forward_features(images) if hasattr(self.vision_tower, "forward_features") else self.vision_tower(images)
        if features.dim() == 4:
            B, C, H, W = features.shape
            features = features.flatten(2).transpose(1, 2)
        elif features.dim() == 2:
            features = features.unsqueeze(1)
        return features

    def to(self, *args, **kwargs):
        super().to(*args, **kwargs)
        if self.vision_tower is not None:
            self.vision_tower = self.vision_tower.to(*args, **kwargs)
        return self

class LlavaMetaModel:
    def __init__(self, config):
        super().__init__(config)
        if hasattr(config, "mm_vision_tower") and config.mm_vision_tower:
            self.vision_tower = MobileCLIPVisionTower(config)
            self.mm_projector = build_vision_projector(config)

    def get_vision_tower(self):
        return getattr(self, "vision_tower", None)

class LlavaMetaForCausalLM(ABC):
    @abstractmethod
    def get_model(self): pass

    def get_vision_tower(self):
        return self.get_model().get_vision_tower()

    def encode_images(self, images):
        vt = self.get_model().get_vision_tower()
        if not vt.is_loaded:
            vt.load_model()
            vt = vt.to(device=images.device, dtype=images.dtype)
        return self.get_model().mm_projector(vt(images))

class LlavaQwen3Model(LlavaMetaModel, Qwen3Model):
    config_class = LlavaQwen3Config
    def __init__(self, config):
        super().__init__(config)

class LlavaQwen3ForCausalLM(Qwen3ForCausalLM, LlavaMetaForCausalLM):
    config_class = LlavaQwen3Config
    def __init__(self, config):
        super(Qwen3ForCausalLM, self).__init__(config)
        self.model = LlavaQwen3Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def get_model(self):
        return self.model

    def forward(self, input_ids=None, attention_mask=None, position_ids=None, past_key_values=None,
                inputs_embeds=None, labels=None, use_cache=None, output_attentions=None,
                output_hidden_states=None, images=None, image_sizes=None, return_dict=None, **kwargs):
        return super().forward(input_ids=input_ids, attention_mask=attention_mask, position_ids=position_ids,
            past_key_values=past_key_values, inputs_embeds=inputs_embeds, labels=labels, use_cache=use_cache,
            output_attentions=output_attentions, output_hidden_states=output_hidden_states, return_dict=return_dict)

    @torch.no_grad()
    def generate(self, inputs=None, images=None, image_sizes=None, **kwargs):
        if images is not None:
            inputs_embeds = self.encode_images(images)
            text_embeds = self.get_model().embed_tokens(inputs)
            inputs_embeds = torch.cat([inputs_embeds, text_embeds], dim=1)
            return super().generate(inputs_embeds=inputs_embeds, **kwargs)
        return super().generate(inputs, **kwargs)

AutoConfig.register("llava_qwen3", LlavaQwen3Config)
AutoModelForCausalLM.register(LlavaQwen3Config, LlavaQwen3ForCausalLM)
'''

with open("/tmp/modeling_llava_qwen.py", "w") as f:
    f.write(modeling_code)

print("Created modeling_llava_qwen.py")

In [ ]:
# ============================================
# UPLOAD TO HUGGINGFACE (with custom code)
# ============================================
from pathlib import Path
from huggingface_hub import HfApi, create_repo

api = HfApi(token=CONFIG["hf_token"])

# Create repo if needed
try:
    create_repo(repo_id=CONFIG["hf_repo"], repo_type="model", exist_ok=True, token=CONFIG["hf_token"])
    print(f"Repository ready: {CONFIG['hf_repo']}")
except Exception as e:
    print(f"Repo exists or error: {e}")

print(f"\nUploading model to {CONFIG['hf_repo']}...")
api.upload_folder(
    folder_path=MERGED_DIR,
    repo_id=CONFIG["hf_repo"],
    repo_type="model",
    commit_message="Upload Belle-VLM model weights",
)
print("Model uploaded!")

# Upload local custom code files
required_files = [
    "configuration_llava_qwen.py",
    "modeling_llava_qwen.py",
    "fastvit.py",
]

candidate_dirs = [
    Path("."),
    Path("hf_upload"),
    Path("../hf_upload"),
    Path("ml-fastvlm/hf_upload"),
    Path("/kaggle/working/ml-fastvlm-v2/hf_upload"),
    Path("/kaggle/working/ml-fastvlm/hf_upload"),
]

resolved = {}
for name in required_files:
    found = None
    for d in candidate_dirs:
        p = d / name
        if p.exists():
            found = p
            break
    if found is None:
        raise FileNotFoundError(f"Missing required custom file: {name}. Tried: {candidate_dirs}")
    resolved[name] = found

print("\nUploading custom code files...")
for name, path in resolved.items():
    api.upload_file(
        path_or_fileobj=str(path),
        path_in_repo=name,
        repo_id=CONFIG["hf_repo"],
    )
    print(f"Uploaded {name} from {path}")

print(f"\n{'='*50}")
print("Model uploaded successfully!")
print(f"https://huggingface.co/{CONFIG['hf_repo']}")
print(f"{'='*50}")



## 8. Test Inference

In [ ]:
# ============================================
# TEST INFERENCE
# ============================================
import sys
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPImageProcessor

sys.path.insert(0, '/kaggle/working/ml-fastvlm-v2')

from llava.conversation import conv_templates
from llava.mm_utils import tokenizer_image_token, process_images
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.model.language_model.llava_qwen import LlavaQwen3ForCausalLM

HF_MODEL_PATH = CONFIG["hf_repo"]

print(f"Loading model from: {HF_MODEL_PATH}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = LlavaQwen3ForCausalLM.from_pretrained(
    HF_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

# Setup vision tower
vision_tower = model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower = vision_tower.to(device=model.device, dtype=torch.float16)
image_processor = vision_tower.image_processor

print(" Model loaded!")

In [ ]:
# Test with dataset
from datasets import load_dataset
from IPython.display import display

test_dataset = load_dataset(CONFIG["dataset_name"], split="train")
test_image = test_dataset[0]['image']
test_question = test_dataset[0]['vi_problem']

display(test_image.resize((300, 300)))
print(f"\nQuestion: {test_question[:200]}...")

# Process
if test_image.mode != 'RGB':
    test_image = test_image.convert('RGB')

image_tensor = process_images([test_image], image_processor, model.config)[0]

conv = conv_templates["qwen_3"].copy()
conv.append_message(conv.roles[0], f"{DEFAULT_IMAGE_TOKEN}\n{test_question}")
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)

# Generate
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor.unsqueeze(0).to(dtype=torch.float16, device=model.device),
        image_sizes=[test_image.size],
        do_sample=True,
        temperature=0.7,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

print("="*60)
print("MODEL RESPONSE:")
print("="*60)
print(output)

## Done!

Model trained with **EPOCHS** and uploaded to HuggingFace.

View: https://huggingface.co/beyoru/Belle-VLM